In [1]:
%cd "C:\Users\andrewmauro\Desktop\springboard\Project Excercises\Kaggle - Mercari Price Suggestion"

C:\Users\andrewmauro\Desktop\springboard\Project Excercises\Kaggle - Mercari Price Suggestion


Data Ingestion, Wrangling, and Manipulation

Purpose: 

To prepare the Mercari Price dataset for visualization by cleaning and extracting additional information from text columns, and calculating average prices based on grouped categorical features. In addition, Natural Language processing will be performed to obtain additional features for prediction purposes.

Procedures

The following data wrangling steps were performed:

1. The 'category_name' field was split into five levels of category names, with each level increasing in specificity.
2. NULL values in all columns were replaced with 'Unknown'
3. Averages were calculated, grouping by each categorical variable (with the exception of item description)
4. Natural Language processing was performed on the 'item_description' field. The text was normalized to stem, lemmatize all words in each item descritpion and remove stop words. The following additional features were derived for prediction:
(a) Bag of Words count
(b) Term frequency over individual document frequency ratio


Section 1 - Data Ingestion

The below code will import the training set and then perform basic data manipulation to obtain a dataset with some additional category features. 

In [2]:
#Section 1.1 - Import packages and Dataset
import pandas as pd
import numpy as np

#text analytics
#regular expressions
import re

price = pd.read_table(filepath_or_buffer = 'train.tsv', sep = '\t', index_col = 'train_id')

#check data set size
#mb = 329893 / 1024
#print(mb) #322 MB
#We noted that our data set is very large

#create sample set
price = pd.DataFrame.sample(price, n = 100)


C:\Users\andrewmauro\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\lib\arraysetops.py:463: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
price.shape


(100, 7)

In [4]:

price.head(10)

,name,item_condition_id,category_name,brand_name,price,shipping,item_description
train_id,,,,,,,
367003,Sexy tie-up Crop Top,2,"Women/Tops & Blouses/Tank, Cami",NaN,16.0,0,No description yet
432117,VS PINK ULTIMATE SPORTS BRAS M,2,Women/Athletic Apparel/Sports Bras,PINK,49.0,0,One is NWT others were gently used in great co...
292211,Air jordans,2,Women/Shoes/Athletic,NaN,59.0,0,Only worn about 2 times. I am a size 8 in wome...
1389106,Barbicide jars new never used in box,1,Beauty/Tools & Accessories/Hair Styling Tools,NaN,18.0,1,For salon or barber use price is 15$ for one o...
1377933,Mossimo,3,"Women/Athletic Apparel/Pants, Tights, Leggings",Mossimo Supply Co.,8.0,1,Pre loved size M
405939,SIZE 9-13 MEN's black TITANIUM ring,1,Men/Other/Other,NaN,19.0,1,.Available size :9-13 .Crafted in TITANIUM .Ta...
639123,Tom Ford Black Orchid Mini Roller,2,Beauty/Fragrance/Women,Tom Ford,10.0,1,Used once
660243,Tech21 Evo mesh burgundy iPhone 6/6s,1,"Electronics/Cell Phones & Accessories/Cases, C...",NaN,14.0,0,Brand new Tech21 Evo mesh burgundy iPhone 6/6s
884106,Victoria's Secret flip flops,1,Women/Shoes/Sandals,Victoria's Secret,13.0,1,Brand new in unopened packaging Victoria's Sec...


Section 2 - Data Wrangling, Maniuplation, and Natural Language Processing

The below code will create additional features from the training set data for the purposes of visualization and predictive modeling. All text columns will be converted to lower case text, and the category name column will be split into five high-level categories, with each successive category level offering greater specificity as to the item type.

Additionally, null values in our text will be replaced with "unknown," or in the case of the item description category, "no description yet."

We will also calculate mean prices for categorical variables.

In [5]:
#Section 2.1 - Data Wrangling and Maniulation - Split category name column, Remove NULLs and then 
##add columns that contain category averages

#switch all text columns to lower
price[['name']] = price['name'].str.lower()
price[['category_name']] = price['category_name'].str.lower()
price[['brand_name']] = price['brand_name'].str.lower()
price[['item_description']] = price['item_description'].str.lower()

#split category name column
price[['catOne','catTwo', 'catThree', 'catFour', 'catFive']] = price['category_name'].str.split('/',expand=True)

#replace empty brand names and category names with 'Unknown'

price['brand_name'][(price['brand_name'].isnull())] = 'unknown'
price['catOne'][(price['catOne'].isnull())] = 'unknown'
price['catTwo'][(price['catTwo'].isnull())] = 'unknown'
price['catThree'][(price['catThree'].isnull())] = 'unknown'
price['catFour'][(price['catFour'].isnull())] = 'unknown'
price['catFive'][(price['catFive'].isnull())] = 'unknown'
price['item_description'][(price['item_description'].isnull())] = 'no description yet'

#obtain average prices grouping by categorical variables
price['meanCondition'] = price.groupby('item_condition_id').price.transform('mean')
price['meanBrand'] = price.groupby('brand_name').price.transform('mean')
price['countBrand'] = price.groupby('brand_name').price.transform('count')
price['meanShipping'] = price.groupby('shipping').price.transform('mean')

price['meanLevelOne'] = price.groupby('catOne').price.transform('mean')
price['countLevelOne'] = price.groupby('catOne').price.transform('count')

price['meanLevelTwo'] = price.groupby(['catOne', 'catTwo']).price.transform('mean')
price['countLevelTwo'] = price.groupby(['catOne', 'catTwo']).price.transform('count')

price['meanLevelThree'] = price.groupby(['catOne', 'catTwo', 'catThree']).price.transform('mean')
price['countLevelThree'] = price.groupby(['catOne', 'catTwo', 'catThree']).price.transform('count')

price['meanLevelFour'] = price.groupby(['catOne', 'catTwo', 'catThree', 'catFour']).price.transform('mean')
price['countLevelFour'] = price.groupby(['catOne', 'catTwo', 'catThree', 'catFour']).price.transform('count')

price['meanLevelFive'] = price.groupby(['catOne', 'catTwo', 'catThree', 'catFour', 'catFive']).price.transform('mean')
price['countLevelFive'] = price.groupby(['catOne', 'catTwo', 'catThree', 'catFour', 'catFive']).price.transform('count')

price['lengthDescription'] = price['item_description'].str.len()

#fields for visualization
#price.loc[:, ['item_condition_id', 'brand_name', 'shipping', 'catOne', 'catTwo', 'meanBrand', 'countBrand', 'meanLevelOne', 'countLevelOne', 'meanLevelTwo', 'countLevelTwo', 'price']].to_csv('trainViz.csv')

#select sample for analyis below
#price = pd.DataFrame.sample(price, n = 300)


C:\Users\andrewmauro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app
C:\Users\andrewmauro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
C:\Users\andrewmauro\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-vers

In [6]:
#Review Cleaned Dataset
price.head(5)

,name,item_condition_id,category_name,brand_name,price,shipping,item_description,catOne,catTwo,catThree,...,countLevelOne,meanLevelTwo,countLevelTwo,meanLevelThree,countLevelThree,meanLevelFour,countLevelFour,meanLevelFive,countLevelFive,lengthDescription
train_id,,,,,,,,,,,,,,,,,,,,,
367003,sexy tie-up crop top,2,"women/tops & blouses/tank, cami",unknown,16.0,0,no description yet,women,tops & blouses,"tank, cami",...,46.0,25.142857,7.0,16.000000,1.0,16.000000,1.0,16.000000,1.0,18
432117,vs pink ultimate sports bras m,2,women/athletic apparel/sports bras,pink,49.0,0,one is nwt others were gently used in great co...,women,athletic apparel,sports bras,...,46.0,25.187500,16.0,28.333333,3.0,28.333333,3.0,28.333333,3.0,99
292211,air jordans,2,women/shoes/athletic,unknown,59.0,0,only worn about 2 times. i am a size 8 in wome...,women,shoes,athletic,...,46.0,29.200000,5.0,59.000000,1.0,59.000000,1.0,59.000000,1.0,112
1389106,barbicide jars new never used in box,1,beauty/tools & accessories/hair styling tools,unknown,18.0,1,for salon or barber use price is 15$ for one o...,beauty,tools & accessories,hair styling tools,...,13.0,18.000000,1.0,18.000000,1.0,18.000000,1.0,18.000000,1.0,79
1377933,mossimo,3,"women/athletic apparel/pants, tights, leggings",mossimo supply co.,8.0,1,pre loved size m,women,athletic apparel,"pants, tights, leggings",...,46.0,25.187500,16.0,26.111111,9.0,26.111111,9.0,26.111111,9.0,16


Section 3 - Natural Language Processing Pre-processing Steps

We will adjust the item description column to:

(1) Include only alphabetic characters from our text
(2) Lemmatize all words (identify the appropriate part of speech based on context and group similar words together)

In [7]:
#Section 3.1 - import packages
import re
import nltk

#include only alphabetic characters
price[['item_description']] = price['item_description'].str.replace('[^a-zA-Z\s]+', '')

#lemmatize

#define lemmatizer function
toke = nltk.tokenize.WhitespaceTokenizer()
wnl = nltk.stem.wordnet.WordNetLemmatizer()
def lemmatize_text(text):
   return [wnl.lemmatize(w) for w in toke.tokenize(text)]

price[['item_description']] = price['item_description'].apply(lemmatize_text)
price[['item_description']] = price['item_description'].apply(lambda x: ' '.join(x))

#review
#print(price[['item_description']].head())


In [8]:
print(price.item_description.values)

['no description yet'
 'one is nwt others were gently used in great condition all size medium push up ultimate sport bra'
 'only worn about time i am a size in woman and they fit perfectly size display is y price is negotiable'
 'for salon or barber use price is for one or for new in box never used'
 'pre loved size m'
 'available size crafted in titanium tarnish resistance diamond accented men black titanium ring with bright stone'
 'used once' 'brand new tech evo mesh burgundy iphone s'
 'brand new in unopened packaging victoria secret flip flop size small run a little large im a size to shoe and these seemed too large'
 'some used once some great condition some good condition but no flaw all onesies except the one on the far left in first pic and wild man is a tank selling a a bundle only'
 'assorted baby boy clothes for fallwinter size are ranging from newborn to month old pc are newborn pc are pc are month need a fresh wash a these ha been stored in bin for year some ha minor fuss

Section 3.2

NLP Feature #1 - Bag of Words Vectors

Our first NLP feature is a "bag of words" vector depicting an identifier for each word in each cell of the item_description column, and corresponding vectors depicting the total word count for each word within each cell. The occurence of particular words can be used as a predictor for determining item price.

Fit/Transform

We do this through the fit/transform method. This will call the models fit and transform emthods. THis is what helps us map word ids to vectors depicting each word's occurenccce. Fit will find parameters or norms in the data, and transform will apply the model's underlying algorithm or approximation, similar to pre-processing but with a specific use case in mind.

In [9]:
#Section 3.2 - Natural Language Processing - Bag of Words

#import pacakages
from sklearn.feature_extraction.text import CountVectorizer

#create word count vectorizer - test with full data set, and then use a sample
countVectorizer = CountVectorizer(stop_words = 'english')

countTrain = countVectorizer.fit_transform(price.item_description.values)


In [10]:
#Obtain the first ten features of the count vector
#countVectorizer.get_feature_names()[:10]

#countDf = pd.DataFrame(countTrain.A, columns = countVectorizer.get_feature_names())

#print(countDf.shape)
#print(countDf)

Section 3.3

NLP Feature #2 - Tf - idf Feature

A metric that indicates the term frequency within a given item description relative to the frequency of that term in all item descriptions may be predictive of the given item's price. We will use the TfidfVectorizer function from the sklearn library to derive this feature for our analysis.



In [11]:
#Section 3.3. - NLP Bag of Words

from sklearn.feature_extraction.text import TfidfVectorizer 

# Initialize a TfidfVectorizer object: tfidfVectorizer
tfidfVectorizer = TfidfVectorizer(stop_words = "english", max_df = 0.7)

# Transform the training data: tfidf_train 
tfidfTrain = tfidfVectorizer.fit_transform(price.item_description.values)

# Print the first 10 features
print(tfidfVectorizer.get_feature_names()[:10])

# Print the first 5 vectors of the tfidf training data
print(tfidfTrain.A[:5])


['accented', 'accepted', 'actually', 'addition', 'adidas', 'adjusted', 'ae', 'alphalete', 'amazing', 'american']
[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]


In [12]:
tfidfDF = pd.DataFrame(tfidfTrain.A, columns = tfidfVectorizer.get_feature_names())
#print(tfidfDF)

Section 4 - Conclusion of Analysis and Saving of Data Wrangled Datasets

In [13]:
#Section 4 - Save a local copy of data frames for prediction
from sklearn.model_selection import train_test_split

#create train set and holdout set
#X_train, X_test, y_train, y_test = train_test_split(price["price"], y, test_size = 0.25, random_state = 56)

#write to csv
price.to_csv('priceWrangle.csv')


#X_train.to_csv('trainPredictors.csv')
#y_train.to_csv('trainOutcomes.csv')
#X_test.to_csv('testPredictors.csv')
#y_test.to_csv('testOutcomes.csv')

Conclusion: We have successfully wrangled and manipulated our data frame of prices. We have additional predictor variables related to High Level item categories, High Level item category average prices, item description length. We have also applied natural language processing to the item description category to obtain word counts and term frequency relative to document frequency predictors.